- (Gaussian) Multinomial Naive Bayes with Grid Search
- Logistic Regression
- Support Vector Machines
- Decision Tree
- Random Forest & Bagging Classifier
- Boosting XGBoost
- Neural Nets



In [30]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import sklearn.datasets as datasets
from sklearn.kernel_approximation import Nystroem
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
import pandas as pd
from collections import Counter
from scipy.stats import reciprocal, uniform

In [2]:
from LanguageModels.Word2Vec import Word2Vec
from Preprocessing.LemmatizerPreprocessor import LemmatizerPreprocessor
from Preprocessing.DataLoader import DataLoader

[nltk_data] Downloading package wordnet to /home/ckmj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/ckmj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ckmj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ckmj/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Load Dataset

In [3]:
'''
file = open("../data/processed/data.p",'rb')
data = pickle.load(file)
file.close()

file = open("../data/processed/labels.p",'rb')
labels = pickle.load(file)
file.close()
'''
#Using data pipeline

data = DataLoader('../data/EMNLP2020.csv').load()

# Load preprocessor
lp = LemmatizerPreprocessor()

In [4]:
w2v = Word2Vec(path='../data/glove.6B/glove.6B.200d.txt')

In [5]:
# multiclass case
data, label = w2v.featurize(data, lp, mode='multiclass')
print(len(data))
print(len(label))
label = label.ravel()


1450
1450


In [56]:
'''
label = labels[:,0]
print('embeddings: {}'.format(data.shape))
print('label: {}'.format(label.shape))
'''

"\nlabel = labels[:,0]\nprint('embeddings: {}'.format(data.shape))\nprint('label: {}'.format(label.shape))\n"

In [57]:
# get rid of Nones
# for i in range(label.shape[0]):
#     print(label[i])
#     if label[i] == None:
#         print(i)
# 299 -> 448

In [58]:
'''
label = np.concatenate((label[:298], label[449::]), 0)
label=label.astype('int')
data = np.concatenate((data[:298], data[449::]), 0)
print('embeddings: {}'.format(data.shape))
print('label: {}'.format(label.shape))
'''

"\nlabel = np.concatenate((label[:298], label[449::]), 0)\nlabel=label.astype('int')\ndata = np.concatenate((data[:298], data[449::]), 0)\nprint('embeddings: {}'.format(data.shape))\nprint('label: {}'.format(label.shape))\n"

### Support Vector Machine: Multi-Class Classification

In [59]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42, stratify=label)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1160, 200) (290, 200) (1160,) (290,)


In [60]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

### Test on Training Data

In [61]:
pred_train = clf.predict(X_train)
accuracy_score(y_train, pred_train)

0.881896551724138

In [62]:
confusion_matrix(y_train, pred_train)

array([[ 50,   4,   1,   4,   2,   0,   0,   2,   0,   0,   2],
       [  2, 117,   0,   2,   5,   0,   0,   0,   0,   4,   2],
       [  0,   2,  23,   1,   1,   0,   0,   0,   0,   2,   0],
       [  2,   6,   0,  93,   1,   0,   0,   3,   0,   2,   4],
       [  0,   2,   0,   3,  44,   0,   0,   0,   0,   3,   0],
       [  1,   4,   0,   4,   1,  29,   0,   2,   0,   0,   4],
       [  1,   1,   0,   1,   0,   0,  16,   1,   0,   1,   1],
       [  1,   3,   0,  10,   0,   1,   0,  37,   0,   2,   3],
       [  1,   1,   0,   1,   0,   1,   0,   0,   4,   0,   1],
       [  1,  10,   0,   5,   1,   0,   0,   1,   0,  57,   3],
       [  1,   2,   0,   3,   0,   0,   0,   1,   0,   0, 553]])

### Test on Testing Data

In [63]:
pred_test = clf.predict(X_test)
accuracy_score(y_test, pred_test)

0.6862068965517242

In [64]:
confusion_matrix(y_test, pred_test)

array([[  7,   3,   0,   3,   0,   0,   0,   1,   0,   1,   1],
       [  2,  22,   1,   4,   1,   0,   0,   0,   0,   1,   2],
       [  0,   0,   0,   1,   2,   0,   0,   2,   0,   1,   1],
       [  3,   5,   0,  16,   0,   0,   0,   1,   0,   1,   2],
       [  0,   2,   0,   1,   9,   0,   0,   0,   0,   0,   1],
       [  2,   3,   0,   2,   1,   0,   0,   1,   0,   1,   1],
       [  1,   0,   0,   1,   1,   0,   1,   0,   0,   2,   0],
       [  1,   4,   0,   4,   0,   0,   0,   4,   0,   0,   1],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   0],
       [  0,   7,   0,   2,   1,   0,   1,   0,   0,   9,   0],
       [  2,   0,   0,   2,   0,   1,   1,   2,   0,   1, 131]])

## Linear Regression & Evaluation of Different Types of Regularization

## Vanilla Linear Regression (No Regularization)

In [65]:
reg = LinearRegression().fit(X_train, y_train)
print("Training Accuracy: " + str(reg.score(X_train, y_train)))
print("Training Accuracy: " + str(reg.score(X_test, y_test)))

Training Accuracy: 0.6516076647956162
Training Accuracy: 0.4199589511915177


## Ridge Regression (L2 Regularization)

In [66]:
parameters = {'alpha':[1e-4, 1e-3, 1e-2, 0.1, 0.2, 0.5, 1, 2, 5, 10]}
rid = Ridge()
clf = GridSearchCV(rid, parameters, cv=10, verbose=1)
clf.fit(X_train, y_train)
print("Training Accuracy: " + str(clf.score(X_train, y_train)))
print("Testing Accuracy: " + str(clf.score(X_test, y_test)))
print(clf.best_params_)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Training Accuracy: 0.6097467533045435
Testing Accuracy: 0.4799692156646369
{'alpha': 2}


## Lasso Regression (L1 Regularization)

In [67]:
parameters = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.2, 0.5, 1, 2, 5, 10, 100]}

las = Lasso(max_iter = 10000)
clf = GridSearchCV(las, parameters, cv=10, verbose=1)
clf.fit(X_train, y_train)
print("Training Accuracy: " + str(clf.score(X_train, y_train)))
print("Testing Accuracy: " + str(clf.score(X_test, y_test)))
print(clf.best_params_)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Training Accuracy: 0.6315164056965387
Testing Accuracy: 0.4388452721328957
{'alpha': 0.001}


## ElasticNet Regression (L1 + L2 Regularization)

In [68]:
parameters = {'alpha':[1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.2, 0.5, 1, 2, 5, 10, 100], 'l1_ratio':[0.05, 0.1, 0.15, 0.25, 0.5, 0.75, 1]}

las = ElasticNet(max_iter = 10000)
clf = GridSearchCV(las, parameters, cv=10, verbose=1)
clf.fit(X_train, y_train)
print("Training Accuracy: " + str(clf.score(X_train, y_train)))
print("Testing Accuracy: " + str(clf.score(X_test, y_test)))
print(clf.best_params_)

Fitting 10 folds for each of 84 candidates, totalling 840 fits


KeyboardInterrupt: 

## Train/Test on Multiple Classifiers

In [45]:
seeds = [42 * i for i in range(1)]
parameters = {
    'SVM': {'kernel': ['linear', 'rbf', 'poly', 'sigmoid'], 'C':[0.01, 0.1, 1], 'gamma': ['scale', 'auto']},
    # 'GaussianProcess': {'kernel': [1.0 * RBF(1.0)]},
    'LogisticRegression': {'random_state': [0], 'max_iter': [10000]},
    'DecisionTree': {'max_depth': [5, 15, None], 'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random'], 'max_features':[None, 'auto'], 'min_weight_fraction_leaf': [0, 0.1]},
    'RandomForest': {'max_depth': [5, 15, None], 'n_estimators': [10, 100], 'max_features': [None]},
    'MLP': {'alpha': [0.001, 0.01, 0.1], 'max_iter': [10000], 'early_stopping': [False, True], 'beta_2': [0.999, 1]},
    'AdaBoost': {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.5, 1]},
    'NaiveBayes': {},
    'BaggingClassifier': {'base_estimator': [SVC()], 'n_estimators': [10], 'random_state': [0]},
    'GradientBoostingClassifier': {'learning_rate': [0.1, 0.2, 0.5, 1], 'n_estimators': [100, 200, 500, 1000], 'max_depth': [3, 6, 10, 15], 'max_features': ['auto', None], 'n_iter_no_change': [10], 'ccp_alpha': [0, 0.1]}
}

classifiers = {
    'SVM': SVC(),
    # 'GaussianProcess': GaussianProcessClassifier(),
    'LogisticRegression': LogisticRegression(),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'MLP': MLPClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'NaiveBayes': GaussianNB(),
    'BaggingClassifier': BaggingClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
}

In [37]:
import warnings
warnings.filterwarnings('ignore')

In [46]:
# names = ["Linear SVM", "RBF SVM", "Gaussian Process", "Logistic Regression"
#          "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
#          "Naive Bayes", "Bagging Classifier"]

max_acc = []
for seed in seeds:
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=seed, stratify=label)
    score_dict = {}
    for name in classifiers.keys():
        # print('\n*** Training {} ***'.format(name))
        clf = GridSearchCV(classifiers[name], parameters[name], cv=10, verbose=1)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print('Best Params: {}, Test Score: {}'.format(clf.best_params_, score))
        score_dict[name] = score
    max_acc.append(max(score_dict, key=score_dict.get))
    print(score_dict)
    
print(sum(max_acc) / len(max_acc))

Fitting 10 folds for each of 24 candidates, totalling 240 fits
Best Params: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}, Test Score: 0.6655172413793103
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best Params: {'max_iter': 10000, 'random_state': 0}, Test Score: 0.6689655172413793
Fitting 10 folds for each of 48 candidates, totalling 480 fits
Best Params: {'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'min_weight_fraction_leaf': 0.1, 'splitter': 'best'}, Test Score: 0.5275862068965518
Fitting 10 folds for each of 6 candidates, totalling 60 fits
Best Params: {'max_depth': None, 'max_features': None, 'n_estimators': 100}, Test Score: 0.5862068965517241
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Best Params: {'alpha': 0.1, 'beta_2': 0.999, 'early_stopping': False, 'max_iter': 10000}, Test Score: 0.6482758620689655
Fitting 10 folds for each of 9 candidates, totalling 90 fits


KeyboardInterrupt: 

## Rebalancing the Dataset

In [18]:
oversample = SMOTE()
def f(i):
    if i == 11:
        return 0
    else:
        return 1

a = label.ravel()
print(Counter(a))

data = np.array(data)
label = np.array(label)

a = np.where(label == 11)
print(data.shape)
print(label.shape)

data_pos = np.delete(data, a, axis=0)
label_pos = np.delete(label, a)
print(data_pos.shape)
print(label_pos.shape)

X_train, X_test, y_train, y_test = train_test_split(data_pos, label_pos, test_size=0.2, random_state=57, stratify=label_pos)

X_train_rebal, y_train_rebal = oversample.fit_resample(X_train, y_train)
print(Counter(y_train_rebal.tolist()))

Counter({11: 700, 2: 165, 4: 139, 10: 98, 1: 81, 8: 71, 5: 65, 6: 56, 3: 36, 7: 28, 9: 11})
(1450, 200)
(1450,)
(750, 200)
(750,)
Counter({1: 132, 8: 132, 2: 132, 6: 132, 4: 132, 10: 132, 5: 132, 7: 132, 9: 132, 3: 132})


## Multiclass classification, positive labels only, unbalanced

In [19]:
names = ["Linear SVM", "RBF SVM", "Logistic Regression",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "Bagging Classifier"]
         
classifiers = [
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    # GaussianProcessClassifier(1.0 * RBF(1.0), n_jobs=-1),
    LogisticRegression(random_state=0, max_iter = 5000),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators = 1000, class_weight='balanced'),
    MLPClassifier(alpha=1, max_iter=5000),
    AdaBoostClassifier(),
    GaussianNB(),
    BaggingClassifier(base_estimator=SVC(),
                        n_estimators=10, random_state=0)]

score_dict = dict()
for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        score_dict[name] = score
        print(name)
        print(classification_report(clf.predict(X_test), y_test, zero_division=0))
print(score_dict)

Linear SVM
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       1.00      0.22      0.36       150
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0

    accuracy                           0.22       150
   macro avg       0.10      0.02      0.04       150
weighted avg       1.00      0.22      0.36       150

RBF SVM
              precision    recall  f1-score   support

           1       0.38      0.86      0.52         7
           2       0.61      0.29      0.40        68
           3       0.00      0.00      0.00         0
     

## Multiclass classification, positive labels only, balanced

In [20]:
names = ["Linear SVM", "RBF SVM", "Logistic Regression",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "Bagging Classifier"]
         
classifiers = [
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    # GaussianProcessClassifier(1.0 * RBF(1.0), n_jobs=-1),
    LogisticRegression(random_state=0, max_iter = 5000),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators = 1000, class_weight='balanced'),
    MLPClassifier(alpha=1, max_iter=5000),
    AdaBoostClassifier(),
    GaussianNB(),
    BaggingClassifier(base_estimator=SVC(),
                        n_estimators=10, random_state=0)]

score_dict = dict()
for name, clf in zip(names, classifiers):
        clf.fit(X_train_rebal, y_train_rebal)
        score = clf.score(X_test, y_test)
        score_dict[name] = score
        print(name)
        print(classification_report(clf.predict(X_test), y_test, zero_division=0))
print(score_dict)

Linear SVM
              precision    recall  f1-score   support

           1       0.44      0.44      0.44        16
           2       0.33      0.52      0.41        21
           3       0.43      0.33      0.38         9
           4       0.54      0.56      0.55        27
           5       0.62      0.38      0.47        21
           6       0.18      0.25      0.21         8
           7       0.50      0.50      0.50         6
           8       0.36      0.38      0.37        13
           9       0.50      0.09      0.15        11
          10       0.35      0.39      0.37        18

    accuracy                           0.41       150
   macro avg       0.42      0.38      0.38       150
weighted avg       0.44      0.41      0.41       150

RBF SVM
              precision    recall  f1-score   support

           1       0.50      0.67      0.57        12
           2       0.64      0.33      0.43        64
           3       0.00      0.00      0.00         1
     

In [179]:
# Gradient Boosting Classifier, https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
      max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.913

### Using data pipeline for binary logistic regression

In [37]:
#Using data pipeline

data = DataLoader('../data/EMNLP2020.csv').load()

# Load preprocessor
lp = LemmatizerPreprocessor()

In [38]:
w2v = Word2Vec(path='../data/glove.6B/glove.6B.200d.txt')

In [40]:
# binary case
X, y = w2v.featurize(data, lp, mode='binary')
y = y.ravel()

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1160, 200) (290, 200) (1160,) (290,)


In [ ]:
#logistic regression with binary labels

logReg = LogisticRegression(random_state=0, max_iter = 50000)
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)
logReg.fit(X_train, y_train)

test_training= logReg.score(X_train, y_train)
test_testing = logReg.score(X_test, y_test)

print("training data score", test_training, "\ntesting data score", test_testing)

In [42]:
names = ["Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "Bagging Classifier"]
         
classifiers = [
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0), n_jobs=-1),
    LogisticRegression(random_state=0, max_iter = 50000),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(alpha=1, max_iter=5000),
    AdaBoostClassifier(),
    GaussianNB(),
    BaggingClassifier(base_estimator=SVC(),
                        n_estimators=10, random_state=0)]

score_dict = dict()
for name, clf in zip(names, classifiers):
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        score_dict[name] = score
        print(name)
        print(classification_report(clf.predict(X_test), y_test, zero_division=0))
print(score_dict)

Linear SVM
              precision    recall  f1-score   support

           0       0.94      0.82      0.88       171
           1       0.79      0.92      0.85       119

    accuracy                           0.87       290
   macro avg       0.86      0.87      0.86       290
weighted avg       0.88      0.87      0.87       290

RBF SVM
              precision    recall  f1-score   support

           0       0.90      0.93      0.92       145
           1       0.93      0.90      0.91       145

    accuracy                           0.91       290
   macro avg       0.91      0.91      0.91       290
weighted avg       0.91      0.91      0.91       290

Gaussian Process
              precision    recall  f1-score   support

           0       0.96      0.92      0.94       156
           1       0.91      0.96      0.93       134

    accuracy                           0.94       290
   macro avg       0.94      0.94      0.94       290
weighted avg       0.94      0.94     